In [2]:
import os
import random
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd
import math
from requests_html import HTMLSession

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ale\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ale\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ale\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Escoge 100 archivos al azar del directorio de reviews
reviews_folder_path = "reviews/"  
all_reviews_files = os.listdir(reviews_folder_path)
random_files = random.sample(all_reviews_files, 100)
print(random_files)

['10675_2.txt', '757_3.txt', '6663_1.txt', '1372_1.txt', '10340_1.txt', '7259_4.txt', '976_3.txt', '4288_2.txt', '7332_4.txt', '4809_3.txt', '7940_1.txt', '2009_4.txt', '10984_2.txt', '7869_2.txt', '791_1.txt', '12398_2.txt', '11157_3.txt', '1807_2.txt', '5262_4.txt', '10638_2.txt', '4580_1.txt', '3203_1.txt', '1812_1.txt', '10968_1.txt', '5589_2.txt', '8143_1.txt', '7105_3.txt', '462_2.txt', '7659_2.txt', '2562_4.txt', '5209_1.txt', '2535_2.txt', '525_1.txt', '4028_4.txt', '9878_4.txt', '7168_4.txt', '6427_3.txt', '3107_3.txt', '1817_3.txt', '1410_4.txt', '5401_2.txt', '4683_1.txt', '11901_1.txt', '2478_1.txt', '2866_1.txt', '8100_3.txt', '9667_2.txt', '8285_1.txt', '3376_3.txt', '512_1.txt', '10849_1.txt', '12166_1.txt', '1911_3.txt', '9380_4.txt', '9898_1.txt', '11301_3.txt', '2582_2.txt', '9239_2.txt', '1627_1.txt', '8570_2.txt', '5890_4.txt', '8286_1.txt', '225_2.txt', '9732_3.txt', '4053_4.txt', '4681_4.txt', '12442_2.txt', '5193_3.txt', '8355_2.txt', '9225_1.txt', '566_3.txt', '

In [4]:
with open("reviews_url.txt", "r") as file:
    lines = file.readlines()

urls_random_files = []
for file in random_files:
    first_number = file.split("_")[0]
    id = lines[int(first_number)].replace("/usercomments\n", "")
    urls_random_files.append(id)

In [5]:
titles = []
session = HTMLSession()

for url in urls_random_files:
    respone = session.get(url)
    page_html = respone.html
    title= page_html.find('title')[0].text
    final_title = title.split("-")[0]
    titles.append(final_title)

titles

['End Game (2006) ',
 'The Doodlebops (TV Series 2004–2007) ',
 'The Fairly OddParents (TV Series 2001–2017) ',
 'Trapped: Buried Alive (TV Movie 2002) ',
 'The Bell Jar (1979) ',
 'The Legend of Boggy Creek (1972) ',
 'Fever Pitch (2005) ',
 'Timecop: The Berlin Decision (Video 2003) ',
 'Kill Me Later (2001) ',
 'Necromancy (1972) ',
 'The Private Affairs of Bel Ami (1947) ',
 'The House Next Door (TV Movie 2006) ',
 'The Robber Bride (TV Movie 2007) ',
 'Les couloirs du temps: Les visiteurs II (1998) ',
 '3 Non',
 'Maniac Cop 3: Badge of Silence (1992) ',
 'DarkWolf (Video 2003) ',
 'Zabriskie Point (1970) ',
 'Disappearance (TV Movie 2002) ',
 'Curdled (1996) ',
 'What the #$*! Do We (K)now!? (2004) ',
 'Final Justice (1984) ',
 'Wizards of the Lost Kingdom (1985) ',
 'Elektra (2005) ',
 'Circle of Two (1981) ',
 '1st Bite (2006) ',
 'The Screaming Skull (1958) ',
 'Blonde and Blonder (2008) ',
 'Alone in the Dark (2005) ',
 'George of the Jungle 2 (Video 2003) ',
 'Behind the Wate

In [6]:
# Preprocesamiento de los 100 archivos escogidos
# Incluye eliminar numeros, tildes y caracteres adiciones
# Lematizacion

lemmatized_directory = 'lemmatized_reviews/'
os.makedirs(lemmatized_directory, exist_ok=True)

lemmatizer = WordNetLemmatizer()
    
for file in random_files:
    path = os.path.join(reviews_folder_path, file)
        
    with open(path, 'r') as archivo:
        text = archivo.read()
        text=text.lower()
        text=text.replace("< br / >","")
        text=text.replace("<br />","")
        text=text.replace("´", "'")
        text = text.translate(str.maketrans('', '', string.punctuation))
        tokens = nltk.word_tokenize(text)
        stop_words = set(stopwords.words('english'))
        filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
        filtered = [word for word in filtered_tokens if word.isalpha() and word not in stop_words]
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered]
        lemmatized_text = ' '.join(lemmatized_tokens)
            
        # Create a new file with lemmatized content
        lemmatized_path = os.path.join(lemmatized_directory, file)
        with open(lemmatized_path, "w") as output_file:
            output_file.write(lemmatized_text)

print("Lemmatized files have been created in the 'lemmatized_reviews' directory.")

Lemmatized files have been created in the 'lemmatized_reviews' directory.


In [7]:
documents = []

for filename in random_files:
    file_number = filename.split("_")[1].split('.')[0]

    with open(os.path.join(lemmatized_directory, filename), 'r') as input_file:
        file_content = input_file.read()
        documents.append(file_content)

In [8]:
# Frecuencia por documento vectorizado 
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)
term_frequency = X.toarray()
document_frequency = np.sum(X > 0, axis=0)
document_frequency = document_frequency.tolist()[0]
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
tfidf_vectors = tfidf_matrix.toarray()

In [9]:
feature_names = tfidf_vectorizer.get_feature_names_out()

word_freq_dicts = {}

for random_files_index, random_file in enumerate(random_files):
    name_doc = "Doc"+ str(random_files_index) + "_tf"
    word_freq_per_doc_dict = {}
    for word_index, word in enumerate(feature_names):
        word_freq_per_doc_dict[word] = int(term_frequency[random_files_index][word_index])
    
    word_freq_dicts[random_file] = word_freq_per_doc_dict

freq_df = pd.DataFrame(word_freq_dicts)
freq_df

,10675_2.txt,757_3.txt,6663_1.txt,1372_1.txt,10340_1.txt,7259_4.txt,976_3.txt,4288_2.txt,7332_4.txt,4809_3.txt,...,4694_1.txt,12227_1.txt,10281_1.txt,2316_3.txt,10153_4.txt,580_3.txt,3160_1.txt,6646_1.txt,2018_3.txt,5403_1.txt
ability,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
abilitythere,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
able,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aboard,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zombie,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zombiecannibal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zone,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
zrated,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
total_freq = freq_df.sum(axis=1)
all_metrics_df = pd.DataFrame({"total_tf": total_freq})

all_metrics_df["df"] = document_frequency#df_dict.values()
all_metrics_df['idf'] = np.log10(100 / all_metrics_df['df'])
all_metrics_df['tf_idf'] = all_metrics_df['idf'] * all_metrics_df['total_tf']
all_metrics_df_sorted = all_metrics_df.sort_values(by='total_tf', ascending=False)
normalized_column = normalize(all_metrics_df_sorted[['tf_idf']], norm='l2', axis=0)
all_metrics_df_sorted['normalized_tf_idf'] = normalized_column

all_metrics_df_sorted.head(10)

,total_tf,df,idf,tf_idf,normalized_tf_idf
movie,179,62,0.207608,37.161888,0.114178
film,176,58,0.236572,41.636673,0.127926
one,95,55,0.259637,24.665544,0.075784
like,86,54,0.267606,23.014137,0.070710
bad,63,37,0.431798,27.203291,0.083581
good,61,40,0.397940,24.274341,0.074582
get,56,36,0.443697,24.847060,0.076341
make,55,40,0.397940,21.886700,0.067246
character,53,34,0.468521,24.831617,0.076294
would,52,36,0.443697,23.072270,0.070888


In [11]:
all_metrics_df_new_sorted = all_metrics_df_sorted.sort_values(by='tf_idf', ascending=False)
all_metrics_df_new_sorted.head(10)

,total_tf,df,idf,tf_idf,normalized_tf_idf
film,176,58,0.236572,41.636673,0.127926
movie,179,62,0.207608,37.161888,0.114178
palermo,18,1,2.000000,36.000000,0.110608
missile,16,1,2.000000,32.000000,0.098318
show,43,23,0.638272,27.445703,0.084325
bad,63,37,0.431798,27.203291,0.083581
really,47,29,0.537602,25.267294,0.077632
get,56,36,0.443697,24.847060,0.076341
character,53,34,0.468521,24.831617,0.076294
one,95,55,0.259637,24.665544,0.075784


In [12]:
word_freq_dicts = {}

for random_files_index, random_file in enumerate(random_files):
    name_doc = "Doc"+ str(random_files_index) + "tf_idf"
    word_freq_per_doc_dict = {}
    for word_index, word in enumerate(feature_names):
        word_freq_per_doc_dict[word] = tfidf_vectors[random_files_index][word_index]
    
    word_freq_dicts[random_file] = word_freq_per_doc_dict

freq_df = pd.DataFrame(word_freq_dicts)
freq_df

,10675_2.txt,757_3.txt,6663_1.txt,1372_1.txt,10340_1.txt,7259_4.txt,976_3.txt,4288_2.txt,7332_4.txt,4809_3.txt,...,4694_1.txt,12227_1.txt,10281_1.txt,2316_3.txt,10153_4.txt,580_3.txt,3160_1.txt,6646_1.txt,2018_3.txt,5403_1.txt
ability,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.045417,0.172725
abilitythere,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
able,0.0,0.091637,0.0,0.0,0.0,0.0,0.072221,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
abm,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
aboard,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zombie,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
zombiecannibal,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
zone,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.217246,0.0,0.0,0.0,0.000000,0.000000
zrated,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000


In [28]:
# Preprocess the query_text (replace this with your actual query)
query_text = "movies"

# Calculate the TF-IDF vector for the query
query_vector = tfidf_vectorizer.transform([query_text]).toarray()

from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between the query vector and document vectors
similarities = cosine_similarity(query_vector, tfidf_vectors)

similarities

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [31]:
# Get the indices of the top-N most similar documents
top_indices = np.argsort(similarities[0])[::-1][:20]

top_titles = []
top_file_names = []

for index in top_indices:
    print("Title: ",titles[index], end=" ---- ")
    print("File name: ", random_files[index], end=" ---- ")
    print("Review: ", documents[index])

Title:  The Eden Formula (TV Movie 2006)  ---- File name:  5403_1.txt ---- Review:  gave movie single star impossible give lessscientists developed formula replicating organism laba run warehouse la create trex group industrial spy break steal formula remainder film one endless foot chaseof course trexa rubber puppetgets loose commences wipe cast amazing ability sneak within foot someone without noticing promptly bite head offone cast member escape police car spends remainder film driving aimlessly city superior mental ability cant even operate radio never make attempt drive substation donut shop appears hopelessly lostthe trex wreaks havoc throughout city blazing gun battle buildingscardboard mockupsblowing single police car army anyone else ever show activity must commonplace los angeleswe hope sequel isnt planned
Title:  Fast, Cheap & Out of Control (1997)  ---- File name:  6427_3.txt ---- Review:  altogether found documentary strange really pointless know got award thing personally

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
similarity_df = pd.DataFrame(cosine_similarities)
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.006129,0.052553,0.048931,0.013313,0.031079,0.061811,0.068431,0.006255,0.059567,...,0.024311,0.058535,0.006445,0.026287,0.019741,0.012743,0.029515,0.036182,0.040114,0.047693
1,0.006129,1.000000,0.094875,0.054086,0.024555,0.037180,0.062234,0.005174,0.027473,0.026959,...,0.040847,0.023685,0.001937,0.024151,0.040572,0.007293,0.015591,0.038468,0.027594,0.030115
2,0.052553,0.094875,1.000000,0.038374,0.057769,0.056881,0.061557,0.033763,0.027500,0.050969,...,0.066356,0.049633,0.041778,0.014260,0.072909,0.025656,0.054161,0.036049,0.038956,0.061462
3,0.048931,0.054086,0.038374,1.000000,0.025222,0.038396,0.016192,0.006534,0.023815,0.045034,...,0.019517,0.056677,0.014037,0.031413,0.058676,0.046584,0.011409,0.011755,0.037967,0.032662
4,0.013313,0.024555,0.057769,0.025222,1.000000,0.042708,0.059337,0.089657,0.015376,0.078320,...,0.053723,0.050897,0.035204,0.057595,0.059990,0.058724,0.106324,0.010866,0.035766,0.030538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.012743,0.007293,0.025656,0.046584,0.058724,0.038902,0.050058,0.039936,0.047232,0.066281,...,0.028245,0.034114,0.044673,0.037115,0.054571,1.000000,0.053348,0.032385,0.042758,0.023084
96,0.029515,0.015591,0.054161,0.011409,0.106324,0.043312,0.072526,0.038512,0.041394,0.074295,...,0.026929,0.070163,0.033945,0.050688,0.083806,0.053348,1.000000,0.018318,0.042018,0.035402
97,0.036182,0.038468,0.036049,0.011755,0.010866,0.030699,0.040229,0.000000,0.058697,0.029627,...,0.026337,0.018141,0.013316,0.075960,0.052566,0.032385,0.018318,1.000000,0.004967,0.021278
98,0.040114,0.027594,0.038956,0.037967,0.035766,0.037784,0.045317,0.035482,0.027113,0.054715,...,0.045608,0.059743,0.028069,0.029159,0.043522,0.042758,0.042018,0.004967,1.000000,0.023382


In [20]:
threshold = 0.1  # Ajusta el umbral según tu criterio

similar_pairs = []
for i in range(len(documents)):
    for j in range(i+1, len(documents)):
        if cosine_similarities[i][j] > threshold:
            similar_pairs.append((titles[i], titles[j], cosine_similarities[i][j]))

In [21]:
result_df = pd.DataFrame(similar_pairs, columns=["title1", "title2", "cosine"])
new_result_df = result_df.sort_values(by='cosine', ascending=False)

new_result_df.head(10)

,title1,title2,cosine
71,Dillinger (1973),The Bonfire of the Vanities (1990),0.173140
1,The Fairly OddParents (TV Series 2001–2017),3 Non,0.155185
42,Alone in the Dark (2005),Slashers (Video 2001),0.154045
70,The Vampire Bat (1933),Vampires: Los Muertos (2002),0.153846
17,The House Next Door (TV Movie 2006),Beta House (Video 2007),0.152416
39,Alone in the Dark (2005),Wendigo (2001),0.151120
52,Crimson Tide (1995),Crimson Tide (1995),0.145990
34,Elektra (2005),The Captives (Video 2001),0.139498
18,3 Non,Over Her Dead Body (2008),0.136817
63,Cradle of Fear (Video 2001),S.I.C.K. Serial Insane Clown Killer (Video 2003),0.132190
